This jupyter notebook will help you reading and manipulating MSG data. The MSG data we provided to you have been prepared and post-processed by Daniele Corradini (dcorrad1@uni-koeln.de) For any usage you plan to do of them beyond the summer school project work, please inform him via email. 

To work with MSG data, you need to install some packages. You can do that with this command
# Install packages in Google Colab
!pip install satpy matplotlib cartopy

In [7]:
# imports
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pandas as pd
import satpy
import os

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

### Reading ncdf
With the instruction below, you can read the ncdf files in xarray datasets 

In [ ]:
#access Google Drive folder #
files_dir ='/data/sat/msg/netcdf/parallax/2023/07/'

# Define path and filepattern
filepattern = "20230724-EXPATS-RG.nc"
folder_path=files_dir+'/MyDrive/16_teaching/02_Trento_exercise1_2024/Data_ex1/' #TODO change this line with your directory

# Open the datasets using xarray
ds = xr.open_dataset(folder_path+filepattern) #TODO

ds

### Plot map at given timestamp
With the block below, you can plot channel maps for given timestamps using the plotting routine provided below.

In [ ]:
channels = ['VIS006', 'VIS008','IR_016', 'IR_039', 'IR_087', 'IR_097', 'IR_108', 'IR_120', 'IR_134', 'WV_062', 'WV_073']
channels_unit = ['Reflectances (%)','Reflectances (%)','Reflectances (%)', 'Brightness Temperature (K)', 'Brightness Temperature (K)', 'Brightness Temperature (K)', 'Brightness Temperature (K)', 'Brightness Temperature (K)','Brightness Temperature (K)','Brightness Temperature (K)','Brightness Temperature (K)']
channels_cmaps = ['gray','gray', 'gray', 'cool', 'cool', 'cool', 'cool', 'cool', 'cool', 'cool', 'cool']

extent = lon_min, lon_max, lat_min, lat_max = 5., 16., 42., 51.5

# Select a timestamp
data_ss = ds.isel(time=50)

def plot_single_map(ds, extent, channel, ax, channel_unit, channel_cmap):

    # Get one variable values from the dataset, lat and lon
    ch_data = '...'  #TODO insert the right name of the variable
    lat = '...' #TODO insert the right name of the variable
    lon = '...' #TODO insert the right name of the variable

    # Plot the channel data
    ch_plot = ax.pcolormesh(lon, lat, ch_data.values.squeeze(),
                                transform=ccrs.PlateCarree(), cmap=channel_cmap, shading='nearest')

    # Add borders and coastlines
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Add colorbar
    cbar = plt.colorbar(ch_plot, ax=ax, orientation='horizontal', shrink=0.7)
    cbar.set_label(channel_unit)

    # Set extension
    ax.set_extent(extent)

    #set axis thick labels
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=0.75, color='gray', alpha=0.6, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlines = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 10, 'color': 'black'}
    gl.ylabel_style = {'size': 10, 'color': 'black'}

    # Set title
    ax.set_title(f'Channel {channel}')

    return ch_plot


# Define figure structure
num_rows = 4
num_cols = 3
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15), subplot_kw={'projection': ccrs.PlateCarree()})
axs = axs.flatten()

# Plot each channel
for i, channel in enumerate(channels):
    plot_single_map('...') #TODO insert the right fucntion parameters

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

# Show plot
plt.show()

### Nightime filtering
Mask the nighttime timestamps in the dataset for the solar channels

In [ ]:
#Choose time thresholds far from the sunsets and sunrise, considering also the period of the year (Summer!)
#Be careful that the time of the dataset is given in UTC!
time_min = 8 
time_max = 16 

#Choose time thresholds far from the sunsets and sunrise, considering also the period of the year (Summer!)
#Be careful that the time of the dataset is given in UTC!
time_min = 8 #TODO
time_max = 16 #TODO

time_mask = (ds['time.hour'] >= time_min) & (ds['time.hour'] <= time_max) 

# Apply the mask across selected variables
selected_vars = ['IR_016', 'VIS006', 'VIS008'] #TODO
for var in selected_vars:
    ds[var] = ds[var].where(time_mask, np.nan)

#Check if the masking worked

# Compute 25-50-75th percentile and plot them along the time
ds_quantile = ds.quantile([0.25, 0.5, 0.75], dim=['lon', 'lat'])

# Loop through each variable in the dataset
for i, var in enumerate(ds_quantile.data_vars):
    ds_quantile[var].plot.line(x='time', hue='quantile', add_legend=True)
    plt.title(f'Time Trend of Percentiles for {var}')
    plt.xlabel('Time')
    plt.ylabel(channels_unit[i])
    plt.grid(True)
    plt.show()
# Apply the mask across selected variables
selected_vars = ['...'] #TODO insert the name of the channls that are given in Reflectances
for var in selected_vars:
    ds[var] = ds[var].where(time_mask, np.nan)

#Check if the masking worked

# Compute 25-50-75th percentile and plot them along the time
ds_quantile = ds.quantile([0.25, 0.5, 0.75], dim=['lon', 'lat'])

# Loop through each variable in the dataset
for i, var in enumerate(channels):
    ds_quantile[var].plot.line(x='time', hue='quantile', add_legend=True)
    plt.title(f'Time Trend of Percentiles for {var}')
    plt.xlabel('Time')
    plt.ylabel(channels_unit[i])
    plt.grid(True)
    plt.show()

### Crop data
In the code below, you can find an example on how to crop a smaller domain around the Trento area. You can use a similar code with different coordinates to crop the region of the campaign.

In [ ]:
# Coordinates for Trento area
lon_min, lon_max, lat_min, lat_max = 10, 12, 45, 47 #TODO

# Assuming 'ds' is your xarray Dataset and has coordinates named 'lat' and 'lon'
ds_Trento = ds.copy(deep=True)  # Making a deep copy of the dataset

# Compute the conditions TODO
condition = ((ds_Trento.lon >= lon_min) & (ds_Trento.lon <= lon_max) &
             (ds_Trento.lat >= lat_min) & (ds_Trento.lat <= lat_max)).compute()

# Crop the dataset around the specified coordinates
ds_Trento = ds_Trento.where(condition, drop=True) #TODO

## TODO--> check the code before

# Define figure structure
num_rows = 4
num_cols = 3
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15), subplot_kw={'projection': ccrs.PlateCarree()})
axs = axs.flatten()

# Plot each channel
for i, channel in enumerate(channels):
    plot_single_map(ds_Trento.isel(time=50) , extent, channel,axs[i],channels_unit[i],channels_cmaps[i])

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

# Show plot
plt.show()

### Check Outliers

In the code below, you can plot the distribution of the values of each channel to verify if there are outliers, and also filter them out


In [ ]:
 # Funtion that find the maximum and minimum values in a Dataset, for a specified variables
def get_max_min(ds, ch):
    ch_values = ds[ch][:]
    ch_values = ch_values.values.flatten()
    ch_values = ch_values[~np.isnan(ch_values)]
    max = np.amax(ch_values) #TODO
    min = np.amin(ch_values) #TODO

    return min, max


def plot_distribution(channels, data_ss, channels_unit, bin_width):
    num_rows = 4
    num_cols = 3
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))
    axs = axs.flatten()

    # Plot each channel
    for i, channel in enumerate(channels):
        ch_data = data_ss[channel]
        ch_min, ch_max = get_max_min(data_ss, channel)
        bins = np.arange(ch_min, ch_max + bin_width, bin_width) #TODO

        # Effettuare il plot dei dati del canale corrente
        axs[i].hist(ch_data.values.flatten(), bins=bins, density=True)

        axs[i].set_xlabel(channels_unit[i])

        axs[i].set_yscale('log') #TODO

        # Aggiungere il titolo del subplot
        axs[i].set_title(f'Channel {channel}')

    # Aggiusta il layout
    plt.tight_layout()
    fig.subplots_adjust(hspace=0.3, wspace=0.4)

    # Mostra il plot
    plt.show()

plot_distribution(channels, ds_Trento, channels_unit, 1)


# Filter the outliers

value_min = 200 #TODO

selected_vars = ['IR_039'] #TODO

# Apply the mask across selected variables
for var in selected_vars:
    value_mask = ds[var] >= value_min #TODO
    ds[var] = ds[var].where(value_mask, np.nan)

## References

Some papers about satellite remote sensing for precipitation

Inoue, T. (1987). A cloud type classification with NOAA 7 split‐window measurements. Journal of Geophysical Research: Atmospheres, 92(D4), 3991-4000.

Lazri, M., Ameur, S., Brucker, J.M. et al. Identification of raining clouds using a method based on optical and microphysical cloud properties from Meteosat second generation daytime and nighttime data. Appl Water Sci 3, 1–11 (2013). https://doi.org/10.1007/s13201-013-0079-0

Lazri, M., Ameur, S., Brucker, J. M., & Ouallouche, F. (2014). Convective rainfall estimation from MSG/SEVIRI data based on different development phase duration of convective systems (growth phase and decay phase). Atmospheric research, 147, 38-50.

Nauss, T., & Kokhanovsky, A. A. (2006). Discriminating raining from non-raining clouds at mid-latitudes using multispectral satellite data. Atmospheric Chemistry and Physics, 6(12), 5031-5036.

Schmetz, J., Tjemkes, S. A., Gube, M., & Van de Berg, L. (1997). Monitoring deep convection and convective overshooting with METEOSAT. Advances in Space Research, 19(3), 433-441.

Strabala, K. I., S. A. Ackerman, and W. P. Menzel, 1994: Cloud Properties inferred from 812-µm Data. J. Appl. Meteor. Climatol., 33, 212–229, https://doi.org/10.1175/1520-0450(1994)033<0212:CPIFD>2.0.CO;2.

Thies, B., Nauss, T., & Bendix, J. (2008). Delineation of raining from non-raining clouds during nighttime using Meteosat-8 data. Meteorol. Appl, 15, 219-230.

Thies, B., Turek, A., Nauss, T., & Bendix, J. (2010). Weather type dependent quality assessment of a satellite-based rainfall detection scheme for the mid-latitudes. Meteorology and atmospheric physics, 107, 81-89.

